In [21]:
import Android as android
from Android.Resources import Resources
import sqlite3
import re

In [2]:
context = dict(android=android.R)
res = Resources(context)

In [3]:
def unpack_attribs(styleable_id):
    styleable_id = filter(lambda x: bool(x), styleable_id)
    array = map(lambda x:res._unpack_pointer(x).attrib, styleable_id)
    return [(x['name'], x.get('format', 'String')) for x in array]

#### Tipo de componente y resid

In [4]:
NS_MAP = "xmlns:map"

def parse_nsmap(file):
    events = "start", "start-ns", "end-ns"
    root = None
    ns_map = []
    for event, elem in ET.iterparse(file, events):
        if event == "start-ns":
            ns_map.append(elem)
        elif event == "end-ns":
            ns_map.pop()
        elif event == "start":
            if root is None:
                root = elem
            elem.set(NS_MAP, dict(ns_map))
    return ET.ElementTree(root)


In [32]:
def get_manifest_attrib_name(tag_name):
    equiv = {
        'compatible-screens':'compatible-screens-screen',
        'manifest':'',
        'uses-permission-sdk-23':'uses-permission'
            }
    tag_name = equiv.get(tag_name, tag_name)
    attrname = 'AndroidManifest' + tag_name.title().replace('-', '')
    assert hasattr(android.R.styleable, attrname), 'No tag description'
    return attrname

In [33]:
def get_manifest_tag_attibs(tag_name):
    attrname = get_manifest_attrib_name(tag_name)
    styleable_id = getattr(android.R.styleable, attrname)
    styleable_id = filter(lambda x: bool(x), styleable_id)
    array = map(lambda x:res._unpack_pointer(x).attrib, styleable_id)
    return [(x['name'], x.get('format', 'String')) for x in array]

In [7]:
def list_db_tables(conn):
    try:
        cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    except:
        pass
    else:
        return [x[0] for x in cursor]
    finally:
        cursor.close()


In [8]:
def get_table_name(tag):
    return  tag.title().replace('-', '')


In [9]:
def list_table_fields(conn, table_name):
    sql = "PRAGMA table_info('%s');" % table_name
    try:
        cursor = conn.execute(sql)
    except:
        pass
    else:
        return [x[1] for x in cursor.fetchall()]
    finally:
        cursor.close()


In [10]:
def create_table(conn, table_name):
    equivTypes = dict(
        integer='INTEGER',
        string='TEXT',
    )
    attribs = get_manifest_tag_attibs(table_name)
    sql = 'CREATE TABLE %s (id INTEGER PRIMARY KEY, parent INTEGER, %s);'
    if table_name == 'manifest':
        sql = sql.replace(', parent INTEGER,', ', package TEXT,')
    columns = ''
#     fcn = lambda x: '%s %s' % (x[0], equivTypes.get(x[1], 'BLOB'))
    fcn = lambda x: '%s %s' % (x[0], 'TEXT')
    columns = map(fcn, attribs)
    columns = ', '.join(columns)
    table_name = table_name.title().replace('-', '')
    sql = sql % (table_name, columns)
    conn.execute(sql)

In [11]:
def update_table(conn, table_name, value_map):
    fields = list_table_fields(conn, table_name)
    to_update = list(value_map.viewkeys() & set(fields))
    column_values = [value_map.get(x) for x in to_update]
    
    sql = 'INSERT INTO %s (%s) VALUES (%s)'
    column_names = ', '.join(to_update)
    column_placer = ', '.join(len(to_update)*['?'])
    sql = sql % (table_name, column_names, column_placer)
    
    try:
        cursor = conn.execute(sql, column_values)
    except:
        pass
    else:
        return cursor.lastrowid
    finally:
        cursor.close()



In [12]:
def fmtTable(alist, headers=None):
    rowFmt = (len(alist[0])*'|{:>10}' + '|')
    fnc = lambda x: rowFmt.format(*x)
    fmtList = map(fnc, alist)
    if headers:
        header = [len(fmtList[0])*'-', 
                  rowFmt.format(*headers), 
                  len(fmtList[0])*'-']
        fmtList = header + fmtList
    return fmtList

In [13]:
def query(db, sqlSeq, values=None):
    sql = ' '. join(sqlSeq)
    args = (sql, )
    if values:
        args += (values,)
    try:
        cursor = db.execute(*args) 
    except:
        pass
    else:
        headers = [x[0] for x in cursor.description]
        table = cursor.fetchall()
        cursor.close()
        if table:
            print '\n'.join(fmtTable(table, headers))
        return table 

In [14]:
manifest_tags = [
    '<action>',
     '<activity>',
     '<activity-alias>',
     '<application>',
     '<category>',
     '<compatible-screens>', # <compatible-screens-screen>
     '<data>',
     '<grant-uri-permission>',
     '<instrumentation>',
     '<intent-filter>',
     '<manifest>', # <>
     '<meta-data>',
     '<path-permission>',
     '<permission>',
     '<permission-group>',
     '<permission-tree>',
     '<provider>',
     '<receiver>',
     '<service>',
     '<supports-gl-texture>',
     '<supports-screens>',
     '<uses-configuration>',
     '<uses-feature>',
     '<uses-library>',
     '<uses-permission>',
     '<uses-permission-sdk-23>', # '<uses-permission>'
     '<uses-sdk>'
]

In [ ]:
for tag_name in manifest_tags:
    try:
        attrname = get_manifest_attrib_name(tag_name[1:-1])
    except AssertionError:
        print tag_name

In [34]:
componentInfo = sorted(get_manifest_tag_attibs('activity'))
componentInfo

[('allowEmbedded', 'boolean'),
 ('allowTaskReparenting', 'boolean'),
 ('alwaysFocusable', 'boolean'),
 ('alwaysRetainTaskState', 'boolean'),
 ('autoRemoveFromRecents', 'boolean'),
 ('banner', 'reference'),
 ('clearTaskOnLaunch', 'boolean'),
 ('colorMode', 'String'),
 ('configChanges', 'String'),
 ('description', 'reference'),
 ('directBootAware', 'boolean'),
 ('documentLaunchMode', 'String'),
 ('enableVrMode', 'string'),
 ('enabled', 'boolean'),
 ('excludeFromRecents', 'boolean'),
 ('exported', 'boolean'),
 ('finishOnCloseSystemDialogs', 'boolean'),
 ('finishOnTaskLaunch', 'boolean'),
 ('hardwareAccelerated', 'boolean'),
 ('icon', 'reference'),
 ('immersive', 'boolean'),
 ('label', 'reference|string'),
 ('launchMode', 'String'),
 ('lockTaskMode', 'String'),
 ('logo', 'reference'),
 ('maxAspectRatio', 'float'),
 ('maxRecents', 'integer'),
 ('multiprocess', 'boolean'),
 ('name', 'string'),
 ('noHistory', 'boolean'),
 ('parentActivityName', 'string'),
 ('permission', 'string'),
 ('persist

In [35]:
flags = [x for x in componentInfo if x[1] == 'boolean']
flags

[('allowEmbedded', 'boolean'),
 ('allowTaskReparenting', 'boolean'),
 ('alwaysFocusable', 'boolean'),
 ('alwaysRetainTaskState', 'boolean'),
 ('autoRemoveFromRecents', 'boolean'),
 ('clearTaskOnLaunch', 'boolean'),
 ('directBootAware', 'boolean'),
 ('enabled', 'boolean'),
 ('excludeFromRecents', 'boolean'),
 ('exported', 'boolean'),
 ('finishOnCloseSystemDialogs', 'boolean'),
 ('finishOnTaskLaunch', 'boolean'),
 ('hardwareAccelerated', 'boolean'),
 ('immersive', 'boolean'),
 ('multiprocess', 'boolean'),
 ('noHistory', 'boolean'),
 ('relinquishTaskIdentity', 'boolean'),
 ('resizeableActivity', 'boolean'),
 ('resumeWhilePausing', 'boolean'),
 ('showForAllUsers', 'boolean'),
 ('showOnLockScreen', 'boolean'),
 ('showWhenLocked', 'boolean'),
 ('singleUser', 'boolean'),
 ('stateNotNeeded', 'boolean'),
 ('supportsPictureInPicture', 'boolean'),
 ('systemUserOnly', 'boolean'),
 ('turnScreenOn', 'boolean'),
 ('visibleToInstantApps', 'boolean')]

In [23]:
fields = [x for x in componentInfo if x[1] != 'boolean']
fields

[('banner', 'reference'),
 ('colorMode', 'String'),
 ('configChanges', 'String'),
 ('description', 'reference'),
 ('documentLaunchMode', 'String'),
 ('enableVrMode', 'string'),
 ('icon', 'reference'),
 ('label', 'reference|string'),
 ('launchMode', 'String'),
 ('lockTaskMode', 'String'),
 ('logo', 'reference'),
 ('maxAspectRatio', 'float'),
 ('maxRecents', 'integer'),
 ('name', 'string'),
 ('parentActivityName', 'string'),
 ('permission', 'string'),
 ('persistableMode', 'String'),
 ('process', 'string'),
 ('recreateOnConfigChanges', 'String'),
 ('rotationAnimation', 'String'),
 ('roundIcon', 'reference'),
 ('screenOrientation', 'String'),
 ('splitName', 'string'),
 ('taskAffinity', 'string'),
 ('theme', 'reference'),
 ('uiOptions', 'String'),
 ('windowSoftInputMode', 'String')]

In [26]:
def constantName(varname):
    varname = re.findall(r'([A-Z]*[a-z]+)', varname)
    return 'FLAG_' + '_'.join(map(lambda x: x.upper(), varname))

In [27]:
name = 'excludeFromRecents'
constantName(name)

'FLAG_EXCLUDE_FROM_RECENTS'

In [37]:
map(lambda x:constantName(x[0]), flags)

['FLAG_ALLOW_EMBEDDED',
 'FLAG_ALLOW_TASK_REPARENTING',
 'FLAG_ALWAYS_FOCUSABLE',
 'FLAG_ALWAYS_RETAIN_TASK_STATE',
 'FLAG_AUTO_REMOVE_FROM_RECENTS',
 'FLAG_CLEAR_TASK_ON_LAUNCH',
 'FLAG_DIRECT_BOOT_AWARE',
 'FLAG_ENABLED',
 'FLAG_EXCLUDE_FROM_RECENTS',
 'FLAG_EXPORTED',
 'FLAG_FINISH_ON_CLOSE_SYSTEM_DIALOGS',
 'FLAG_FINISH_ON_TASK_LAUNCH',
 'FLAG_HARDWARE_ACCELERATED',
 'FLAG_IMMERSIVE',
 'FLAG_MULTIPROCESS',
 'FLAG_NO_HISTORY',
 'FLAG_RELINQUISH_TASK_IDENTITY',
 'FLAG_RESIZEABLE_ACTIVITY',
 'FLAG_RESUME_WHILE_PAUSING',
 'FLAG_SHOW_FOR_ALL_USERS',
 'FLAG_SHOW_ON_LOCK_SCREEN',
 'FLAG_SHOW_WHEN_LOCKED',
 'FLAG_SINGLE_USER',
 'FLAG_STATE_NOT_NEEDED',
 'FLAG_SUPPORTS_PICTURE_IN_PICTURE',
 'FLAG_SYSTEM_USER_ONLY',
 'FLAG_TURN_SCREEN_ON',
 'FLAG_VISIBLE_TO_INSTANT_APPS']

In [15]:
resid = android.R.attr.documentLaunchMode
elem = res._unpack_pointer(resid)
map(lambda x:(x.attrib['name'], x.attrib['value']), list(elem))

[('none', '0'), ('intoExisting', '1'), ('always', '2'), ('never', '3')]

In [ ]:
android.R.integer.

In [ ]:
import os
import sqlite3
import xml.etree.ElementTree as ET
import collections

In [ ]:
path = r'./TestActivity/AndroidManifest.xml'
conn = sqlite3.connect(':memory:')
stack = collections.deque()

In [ ]:
root = ET.parse(path).getroot()
stack.append((None, root))
while stack:
    parent_id, element = stack.popleft()
    items = [(key.split('}')[-1], value) for key, value in element.attrib.items()]
    tag, attrib = element.tag, dict(items)
    table_name = get_table_name(tag)
    if table_name not in list_db_tables(conn):
        create_table(conn, tag)
    if parent_id:
        attrib['parent'] = parent_id
    tagid = update_table(conn, table_name, attrib)
    stack.extend([(tagid, item) for item in element])

## NUEVO INTENTO

In [ ]:
path = r'./TestActivity/AndroidManifest.xml'
conn = sqlite3.connect(':memory:')
stack = collections.deque()

In [ ]:
sql = 'CREATE TABLE manifest_tree (id INTEGER PRIMARY KEY, parent INTEGER, tag_type VARCHAR(30), content TEXT);'
conn.execute(sql)

In [ ]:
root = ET.parse(path).getroot()
package = root.attrib.pop('package')
stack.append((None, root))
while stack:
    parent_id, element = stack.popleft()
    items = [(key.split('}')[-1], value) for key, value in element.attrib.items()]
    tag, attrib = element.tag, dict(items)
    content = ' '.join(['%s="%s"' % x for x in attrib.items()])
    valueMap = dict(parent=parent_id, tag_type=tag, content=content)
    tagid = update_table(conn, 'manifest_tree', valueMap)
    stack.extend([(tagid, item) for item in element])

In [ ]:
sql = ['SELECT * FROM manifest_tree']
result = query(conn, sql)

In [ ]:
sql = ['SELECT t1.id, t1.tag_type FROM manifest_tree t1, manifest_tree t2',
       "WHERE t1.parent=t2.id AND",
       "t1.tag_type='intent-filter' AND",
       "t2.tag_type='activity'"
      ]
result = query(conn, sql)

In [ ]:
sql = ['SELECT * FROM manifest_tree',
       "WHERE parent=?"
      ]
result = query(conn, sql, (5,))

In [ ]:
sql = ['SELECT t1.parent, t1.tag_type FROM manifest_tree t1, manifest_tree t2',
       "WHERE t2.parent = t1.id",
       "AND t2.content IN (?, ?)",
       'GROUP BY t1.parent',
       'HAVING COUNT(*) = 2;'
      ]
result = query(conn, sql, 
               (
                'name="android.intent.action.MAIN"',
                'name="android.intent.category.LAUNCHER"'
               ))

In [ ]:
values = [x[0] for x in result]
sql = ['SELECT * FROM manifest_tree',
       "WHERE id in (%s)" % ', '.join(len(values)*'?')
      ]
dummy = query(conn, sql, values)

===============================================================================
### PRUEBA DE CONTENT PROVIDER

In [1]:
import os
import sqlite3
import xml.etree.ElementTree as ET
import collections

In [2]:
from SystemManager.SystemTablesProvider import SystemTablesProvider
from Android.Uri import Uri

In [3]:
provider = SystemTablesProvider()

In [4]:
path = r'./TestActivity/AndroidManifest.xml'
stack = collections.deque()
uri_str = 'content://com.androidapps.systemmanager/system_components'
uri = Uri.parse(uri_str)

In [ ]:
root = ET.parse(path).getroot()
package = root.attrib.pop('package')
stack.append((None, root))
while stack:

    parent_id, element = stack.popleft()
    items = [(key.split('}')[-1], value) for key, value in element.attrib.items()]
    tag, attrib = element.tag, dict(items)
    content = ' '.join(['%s="%s"' % x for x in attrib.items()])
    valueMap = dict(parent=parent_id, tag_type=tag, content=content)
    valueMap

    item_uri = provider.insert(uri, valueMap) 
    tagid = int(item_uri.getLastPathSegment())

    stack.extend([(tagid, item) for item in element])

In [ ]:
lst_tables = list_db_tables(conn)
lst_tables

In [ ]:
cursor = conn.execute('SELECT * FROM %s' % 'Category')
print cursor.fetchall()
cursor.close()

In [ ]:
list_table_fields(conn, 'Action')

In [ ]:
stack

In [ ]:
conn.close()

In [ ]:
import sqlite3

In [ ]:
customers = [
    (101, 'Jhon Doe', 301, 0),
    (102, 'Bruce Wayne', 302, 101),
    (103, 'James Smith', None, 101)
]

In [ ]:
addresses = [
    (301, '12 Main St., Houstn TX 77001'),
    (302, '1007 Mountain Dr., Gotham NY 10286')
]

In [ ]:
orders = [
    (555, 101, '12/24/09', '$156.78'),
    (556, 102, '12/25/09', '$99.99'),
    (557, 101, '12/26/09', '$75.00')
]

In [ ]:
items = [
    (201, 'Tickle Me Elmo', 'it wants to be tickled'),
    (202, 'District 9 DVD', 'Awesome sci-fi movie'),
    (203, 'Baratag', 'It is very sharp')
]

In [ ]:
items_orders = [
    (555, 201),
    (555, 202),
    (556, 202),
    (556, 203)
]

#### Table creation

In [ ]:
db = sqlite3.connect(':memory:')

In [ ]:
sql = 'CREATE TABLE customers ('
sql += 'customer_id INT AUTO INCREMENT PRIMARY KEY, '
sql += 'customer_name VARCHAR(100)'
sql += ');'
sql1 = sql
sql

In [ ]:
sql = 'CREATE TABLE orders ('
sql += 'order_id INT AUTO INCREMENT PRIMARY KEY, '
sql += 'customer_id INT, '
sql += 'amount DOUBLE, '
sql += 'FOREIGN KEY (customer_id) REFERENCES customers(customer_id)'
sql += ');'
sql2 = sql
sql

In [ ]:
for sql in [sql1, sql2]:
    db.execute(sql)

In [ ]:
orders

In [ ]:
sql = 'INSERT INTO orders (order_id, customer_id, amount) VALUES (?, ?, ?)'
values = [
    (1, 1, 19.99),
    (2, 1, 35.15),
    (3, 3, 17.56),
    (4, 4, 12.34),    
]
db.executemany(sql, values)

In [ ]:
sql = 'SELECT * FROM customers JOIN orders;'
db.execute(sql).fetchall()

In [ ]:
sql = 'SELECT * FROM customers, orders;'
db.execute(sql).fetchall()

In [ ]:
sql = 'SELECT * FROM customers NATURAL JOIN orders;'
db.execute(sql).fetchall()

In [ ]:
sql = 'SELECT * FROM customers JOIN orders WHERE customers.customer_id = orders.customer_id;'
db.execute(sql).fetchall()

In [ ]:
sql = 'SELECT * FROM customers JOIN orders'
sql += ' ON (customers.customer_id = orders.customer_id)'
sql += ' WHERE orders.amount > 15;'
db.execute(sql).fetchall()

In [ ]:
sql = 'SELECT * FROM customers JOIN orders'
sql += ' USING customer_id'
sql += ' WHERE orders.amount > 15;'
db.execute(sql).fetchall()

In [ ]:
sql = 'SELECT * FROM customers LEFT OUTER JOIN orders'
sql += ' ON (customers.customer_id = orders.customer_id)'
db.execute(sql).fetchall()

In [ ]:
sql = 'SELECT * FROM customers LEFT OUTER JOIN orders'
sql += ' ON (customers.customer_id = orders.customer_id'
sql += ' AND orders.amount > 15)'
db.execute(sql).fetchall()

In [ ]:
sql = 'SELECT * FROM orders LEFT OUTER JOIN customers'
sql += ' ON (customers.customer_id = orders.customer_id)'
db.execute(sql).fetchall()

In [ ]:
customers
sql = 'CREATE TABLE self_join (id INT, name VARCHAR(20), fkey INT, ref INT);'
db.execute(sql)

In [ ]:
sql = 'INSERT INTO self_join VALUES (?, ?, ?, ?)'
db.executemany(sql, customers)

In [ ]:
sql = 'SELECT * FROM self_join;'
db.execute(sql).fetchall()

In [ ]:
sql = [
    'SELECT DISTINCT t2.id, t2.name, t1.name',
    'FROM self_join t1, self_join t2',
    'WHERE t1.id = t2.ref'
]
sql = ' '.join(sql)
# sql = 'SELECT DISTINCT t2.id, t2.name, t1.name FROM self_join t1, self_join t2 WHERE t1.id = t2.ref'
db.execute(sql).fetchall()

In [ ]:
sql = [
    "UPDATE OR ABORT self_join SET name='aLEX mONTES'",
    "WHERE self_join.id = 101;"
]
sql = ' '.join(sql)
db.execute(sql)

In [ ]:
sql = [
    'SELECT t1.name, COUNT(*)',
    'FROM self_join t1, self_join t2',
    'WHERE t1.id = t2.ref',
    'GROUP BY t1.name',
    'HAVING COUNT(*) = 2;'
]
result = query(db, sql)

In [ ]:
sql = ['SELECT * FROM self_join;']
result = query(db, sql)

In [ ]:
db.close()

In [ ]:
x = (1, 2, 3, 4)

In [ ]:
(len(x)*'|{:>5}' + '|').format(*x)